In [1]:
import tensorflow as tf 
import keras 
import keras.backend as K
from scipy.signal import resample

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from collections import Counter

from keras import regularizers
from keras.models import Sequential, Model, load_model, model_from_json 
from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Reshape, Concatenate,  Dropout 
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU

def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: float(majority/count) for cls, count in counter.items()}



class Estimator:
    l2p = 0.001
    @staticmethod
    def early_layers(inp, fm = (1,3), hid_act_func="relu"):
        # Start
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        # 1
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)

        return x
    
    @staticmethod
    def late_layers(inp, num_classes, fm = (1,3), act_func="softmax", hid_act_func="relu", b_name="Identifier"):
        # 2
        x = Conv2D(32, fm, padding="same", kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(inp)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = Dropout(0.25)(x)
        
        
        # End
        x = Flatten()(x)
        x = Dense(128, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(32, kernel_regularizer=regularizers.l2(Estimator.l2p), activation=hid_act_func)(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_classes, activation=act_func, name = b_name)(x)

        return x
   
    @staticmethod
    def build(height, width, num_classes, name, fm = (1,3), act_func="softmax",hid_act_func="relu"):
        inp = Input(shape=(height, width, 1))
        early = Estimator.early_layers(inp, fm, hid_act_func=hid_act_func)
        late  = Estimator.late_layers(early, num_classes, fm, act_func=act_func, hid_act_func=hid_act_func)
        model = Model(inputs=inp, outputs=late ,name=name)
        return model

Couldn't import dot_parser, loading of dot files will not be possible.


Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt


def get_ds_infos():
    """
    Read the file includes data subject information.
    
    Data Columns:
    0: code [1-24]
    1: weight [kg]
    2: height [cm]
    3: age [years]
    4: gender [0:Female, 1:Male]
    
    Returns:
        A pandas DataFrame that contains inforamtion about data subjects' attributes 
    """ 

    dss = pd.read_csv("data_subjects_info.csv")
    print("[INFO] -- Data subjects' information is imported.")
    
    return dss

def set_data_types(data_types=["userAcceleration"]):
    """
    Select the sensors and the mode to shape the final dataset.
    
    Args:
        data_types: A list of sensor data type from this list: [attitude, gravity, rotationRate, userAcceleration] 

    Returns:
        It returns a list of columns to use for creating time-series from files.
    """
    dt_list = []
    for t in data_types:
        if t != "attitude":
            dt_list.append([t+".x",t+".y",t+".z"])
        else:
            dt_list.append([t+".roll", t+".pitch", t+".yaw"])

    return dt_list


def creat_time_series(dt_list, act_labels, trial_codes, mode="mag", labeled=True, combine_grav_acc=False):
    """
    Args:
        dt_list: A list of columns that shows the type of data we want.
        act_labels: list of activites
        trial_codes: list of trials
        mode: It can be "raw" which means you want raw data
        for every dimention of each data type,
        [attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)].
        or it can be "mag" which means you only want the magnitude for each data type: (x^2+y^2+z^2)^(1/2)
        labeled: True, if we want a labeld dataset. False, if we only want sensor values.
        combine_grav_acc: True, means adding each axis of gravity to  corresponding axis of userAcceleration.
    Returns: 
        It returns a time-series of sensor data.
    
    """
    num_data_cols = len(dt_list) if mode == "mag" else len(dt_list*3)

    if labeled:
        dataset = np.zeros((0,num_data_cols+7)) # "7" --> [act, code, weight, height, age, gender, trial] 
    else:
        dataset = np.zeros((0,num_data_cols))
        
    ds_list = get_ds_infos()
    
    print("[INFO] -- Creating Time-Series")
    for sub_id in ds_list["code"]:
        for act_id, act in enumerate(act_labels):
            for trial in trial_codes[act_id]:
                fname = 'A_DeviceMotion_data/'+act+'_'+str(trial)+'/sub_'+str(int(sub_id))+'.csv'
                raw_data = pd.read_csv(fname)
                raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
                vals = np.zeros((len(raw_data), num_data_cols))
                
                if combine_grav_acc:
                    raw_data["userAcceleration.x"] = raw_data["userAcceleration.x"].add(raw_data["gravity.x"])
                    raw_data["userAcceleration.y"] = raw_data["userAcceleration.y"].add(raw_data["gravity.y"])
                    raw_data["userAcceleration.z"] = raw_data["userAcceleration.z"].add(raw_data["gravity.z"])
                
                for x_id, axes in enumerate(dt_list):
                    if mode == "mag":
                        vals[:,x_id] = (raw_data[axes]**2).sum(axis=1)**0.5        
                    else:
                        vals[:,x_id*3:(x_id+1)*3] = raw_data[axes].values
                    vals = vals[:,:num_data_cols]
                if labeled:
                    lbls = np.array([[act_id,
                            sub_id-1,
                            ds_list["weight"][sub_id-1],
                            ds_list["height"][sub_id-1],
                            ds_list["age"][sub_id-1],
                            ds_list["gender"][sub_id-1],
                            trial          
                           ]]*len(raw_data))
                    vals = np.concatenate((vals, lbls), axis=1)
                dataset = np.append(dataset,vals, axis=0)
    cols = []
    for axes in dt_list:
        if mode == "raw":
            cols += axes
        else:
            cols += [str(axes[0][:-2])]
            
    if labeled:
        cols += ["act", "id", "weight", "height", "age", "gender", "trial"]
    
    dataset = pd.DataFrame(data=dataset, columns=cols)
    return dataset
#________________________________
#________________________________

def ts_to_secs(dataset, w, s, standardize = False, **options):
    
    data = dataset[dataset.columns[:-7]].values    
    act_labels = dataset["act"].values
    id_labels = dataset["id"].values
    trial_labels = dataset["trial"].values

    mean = 0
    std = 1
    if standardize:
        ## Standardize each sensor’s data to have a zero mean and unity standard deviation.
        ## As usual, we normalize test dataset by training dataset's parameters 
        if options:
            mean = options.get("mean")
            std = options.get("std")
            print("[INFO] -- Test Data has been standardized")
        else:
            mean = data.mean(axis=0)
            std = data.std(axis=0)
            print("[INFO] -- Training Data has been standardized: the mean is = "+str(mean)+" ; and the std is = "+str(std))            

        data -= mean
        data /= std
    else:
        print("[INFO] -- Without Standardization.....")

    ## We want the Rows of matrices show each Feature and the Columns show time points.
    data = data.T

    m = data.shape[0]   # Data Dimension 
    ttp = data.shape[1] # Total Time Points
    number_of_secs = int(round(((ttp - w)/s)))

    ##  Create a 3D matrix for Storing Sections  
    secs_data = np.zeros((number_of_secs , m , w ))
    act_secs_labels = np.zeros(number_of_secs)
    id_secs_labels = np.zeros(number_of_secs)

    k=0
    for i in range(0 , ttp-w, s):
        j = i // s
        if j >= number_of_secs:
            break
        if id_labels[i] != id_labels[i+w-1]: 
            continue
        if act_labels[i] != act_labels[i+w-1]: 
            continue
        if trial_labels[i] != trial_labels[i+w-1]:
            continue
            
        secs_data[k] = data[:, i:i+w]
        act_secs_labels[k] = act_labels[i].astype(int)
        id_secs_labels[k] = id_labels[i].astype(int)
        k = k+1
        
    secs_data = secs_data[0:k]
    act_secs_labels = act_secs_labels[0:k]
    id_secs_labels = id_secs_labels[0:k]
    return secs_data, act_secs_labels, id_secs_labels, mean, std
##________________________________________________________________


ACT_LABELS = ["dws","ups", "wlk", "jog", "std", "sit"]
TRIAL_CODES = {
    ACT_LABELS[0]:[1,2,11],
    ACT_LABELS[1]:[3,4,12],
    ACT_LABELS[2]:[7,8,15],
    ACT_LABELS[3]:[9,16],
    ACT_LABELS[4]:[6,14],
    ACT_LABELS[5]:[5,13],
}

In [3]:
class SSA(object):
    
    __supported_types = (pd.Series, np.ndarray, list)
    
    def __init__(self, tseries, L, save_mem=True):
        """
        Decomposes the given time series with a singular-spectrum analysis. Assumes the values of the time series are
        recorded at equal intervals.
        
        Parameters
        ----------
        tseries : The original time series, in the form of a Pandas Series, NumPy array or list. 
        L : The window length. Must be an integer 2 <= L <= N/2, where N is the length of the time series.
        save_mem : Conserve memory by not retaining the elementary matrices. Recommended for long time series with
            thousands of values. Defaults to True.
        
        Note: Even if an NumPy array or list is used for the initial time series, all time series returned will be
        in the form of a Pandas Series or DataFrame object.
        """
        
        # Tedious type-checking for the initial time series
        if not isinstance(tseries, self.__supported_types):
            raise TypeError("Unsupported time series object. Try Pandas Series, NumPy array or list.")
        
        # Checks to save us from ourselves
        self.N = len(tseries)
        if not 2 <= L <= self.N/2:
            raise ValueError("The window length must be in the interval [2, N/2].")
        
        self.L = L
        self.orig_TS = pd.Series(tseries)
        self.K = self.N - self.L + 1
        
        # Embed the time series in a trajectory matrix
        self.X = np.array([self.orig_TS.values[i:L+i] for i in range(0, self.K)]).T
        
        # Decompose the trajectory matrix
        self.U, self.Sigma, VT = np.linalg.svd(self.X)
        self.d = np.linalg.matrix_rank(self.X)
        
        self.TS_comps = np.zeros((self.N, self.d))
        
        if not save_mem:
            # Construct and save all the elementary matrices
            self.X_elem = np.array([ self.Sigma[i]*np.outer(self.U[:,i], VT[i,:]) for i in range(self.d) ])

            # Diagonally average the elementary matrices, store them as columns in array.           
            for i in range(self.d):
                X_rev = self.X_elem[i, ::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.V = VT.T
        else:
            # Reconstruct the elementary matrices without storing them
            for i in range(self.d):
                X_elem = self.Sigma[i]*np.outer(self.U[:,i], VT[i,:])
                X_rev = X_elem[::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.X_elem = "Re-run with save_mem=False to retain the elementary matrices."
            
            # The V array may also be very large under these circumstances, so we won't keep it.
            self.V = "Re-run with save_mem=False to retain the V matrix."
        
        # Calculate the w-correlation matrix.
        self.calc_wcorr()
            
    def components_to_df(self, n=0):
        """
        Returns all the time series components in a single Pandas DataFrame object.
        """
        if n > 0:
            n = min(n, self.d)
        else:
            n = self.d
        
        # Create list of columns - call them F0, F1, F2, ...
        cols = ["F{}".format(i) for i in range(n)]
        return pd.DataFrame(self.TS_comps[:, :n], columns=cols, index=self.orig_TS.index)
            
    
    def reconstruct(self, indices):
        """
        Reconstructs the time series from its elementary components, using the given indices. Returns a Pandas Series
        object with the reconstructed time series.
        
        Parameters
        ----------
        indices: An integer, list of integers or slice(n,m) object, representing the elementary components to sum.
        """
        if isinstance(indices, int): indices = [indices]
        
        ts_vals = self.TS_comps[:,indices].sum(axis=1)
        return pd.Series(ts_vals, index=self.orig_TS.index)
    
    def calc_wcorr(self):
        """
        Calculates the w-correlation matrix for the time series.
        """
             
        # Calculate the weights
        w = np.array(list(np.arange(self.L)+1) + [self.L]*(self.K-self.L-1) + list(np.arange(self.L)+1)[::-1])
        
        def w_inner(F_i, F_j):
            return w.dot(F_i*F_j)
        
        # Calculated weighted norms, ||F_i||_w, then invert.
        F_wnorms = np.array([w_inner(self.TS_comps[:,i], self.TS_comps[:,i]) for i in range(self.d)])
        F_wnorms = F_wnorms**-0.5
        
        # Calculate Wcorr.
        self.Wcorr = np.identity(self.d)
        for i in range(self.d):
            for j in range(i+1,self.d):
                self.Wcorr[i,j] = abs(w_inner(self.TS_comps[:,i], self.TS_comps[:,j]) * F_wnorms[i] * F_wnorms[j])
                self.Wcorr[j,i] = self.Wcorr[i,j]
    
    def plot_wcorr(self, min=None, max=None):
        """
        Plots the w-correlation matrix for the decomposed time series.
        """
        if min is None:
            min = 0
        if max is None:
            max = self.d
        
        if self.Wcorr is None:
            self.calc_wcorr()
        
        ax = plt.imshow(self.Wcorr,interpolation = 'none')
        plt.xlabel(r"$\tilde{F}_i$")
        plt.ylabel(r"$\tilde{F}_j$")
        plt.colorbar(ax.colorbar, fraction=0.045)
        ax.colorbar.set_label("$W_{i,j}$")
        plt.clim(0,1)
        
        # For plotting purposes:
        if max == self.d:
            max_rnge = self.d-1
        else:
            max_rnge = max
        
        plt.xlim(min-0.5, max_rnge+0.5)
        plt.ylim(max_rnge+0.5, min-0.5)
           

In [4]:
#https://stackoverflow.com/a/45305384/5210098
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
results ={}
sdt = ["rotationRate","userAcceleration"]
mode = "mag"
cga = True # Add gravity to acceleration or not

print("[INFO] -- Selected sensor data types: "+str(sdt)+" -- Mode: "+str(mode)+" -- Grav+Acc: "+str(cga))    
act_labels = ACT_LABELS [0:4]

print("[INFO] -- Selected activites: "+str(act_labels))    
trial_codes = [TRIAL_CODES[act] for act in act_labels]
dt_list = set_data_types(sdt)
dataset = creat_time_series(dt_list, act_labels, trial_codes, mode=mode, labeled=True, combine_grav_acc = cga)
print("[INFO] -- Shape of time-Series dataset:"+str(dataset.shape))    


#*****************
TRAIN_TEST_TYPE = "trial" # "subject" or "trial"
#*****************

if TRAIN_TEST_TYPE == "subject":
    test_ids = [4,9,11,21]
    print("[INFO] -- Test IDs: "+str(test_ids))
    test_ts = dataset.loc[(dataset['id'].isin(test_ids))]
    train_ts = dataset.loc[~(dataset['id'].isin(test_ids))]
else:
    test_trail = [11,12,13,14,15,16]  
    print("[INFO] -- Test Trials: "+str(test_trail))
    test_ts = dataset.loc[(dataset['trial'].isin(test_trail))]
    train_ts = dataset.loc[~(dataset['trial'].isin(test_trail))]

print("[INFO] -- Shape of Train Time-Series :"+str(train_ts.shape))
print("[INFO] -- Shape of Test Time-Series :"+str(test_ts.shape))

# print("___________Train_VAL____________")
# val_trail = [11,12,13,14,15,16]
# val_ts = train_ts.loc[(train_ts['trial'].isin(val_trail))]
# train_ts = train_ts.loc[~(train_ts['trial'].isin(val_trail))]
# print("[INFO] -- Training Time-Series :"+str(train_ts.shape))
# print("[INFO] -- Validation Time-Series :"+str(val_ts.shape))

print("___________________________________________________")
print(train_ts.head())

## This Variable Defines the Size of Sliding Window
## ( e.g. 100 means in each snapshot we just consider 100 consecutive observations of each sensor) 
w = 128 # 50 Equals to 1 second for MotionSense Dataset (it is on 50Hz samplig rate)
## Here We Choose Step Size for Building Diffrent Snapshots from Time-Series Data
## ( smaller step size will increase the amount of the instances and higher computational cost may be incurred )
s = 10
train_data, act_train, id_train, train_mean, train_std = ts_to_secs(train_ts.copy(),
                                                                   w,
                                                                   s,
                                                                   standardize = True)

s = 10
# val_data, act_val, id_val, val_mean, val_std = ts_to_secs(val_ts.copy(),
#                                                           w,
#                                                           s,
#                                                           standardize = True,
#                                                           mean = train_mean, 
#                                                           std = train_std)

s = 10
test_data, act_test, id_test, test_mean, test_std = ts_to_secs(test_ts.copy(),
                                                              w,
                                                              s,
                                                              standardize = True,
                                                              mean = train_mean, 
                                                              std = train_std)

print("[INFO] -- Training Sections: "+str(train_data.shape))
#print("[INFO] -- Validation Sections: "+str(val_data.shape))
print("[INFO] -- Test Sections:  "+str(test_data.shape))


id_train_labels = to_categorical(id_train)
#id_val_labels = to_categorical(id_val)
id_test_labels = to_categorical(id_test)

act_train_labels = to_categorical(act_train)
#act_val_labels = to_categorical(act_val)
act_test_labels = to_categorical(act_test)

[INFO] -- Selected sensor data types: ['rotationRate', 'userAcceleration'] -- Mode: mag -- Grav+Acc: True
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(767660, 9)
[INFO] -- Test Trials: [11, 12, 13, 14, 15, 16]
[INFO] -- Shape of Train Time-Series :(621973, 9)
[INFO] -- Shape of Test Time-Series :(145687, 9)
___________________________________________________
   rotationRate  userAcceleration  act   id  weight  height   age  gender  \
0      1.370498          1.195847  0.0  0.0   102.0   188.0  46.0     1.0   
1      1.141648          1.196990  0.0  0.0   102.0   188.0  46.0     1.0   
2      0.372530          1.117437  0.0  0.0   102.0   188.0  46.0     1.0   
3      1.049628          1.088320  0.0  0.0   102.0   188.0  46.0     1.0   
4      0.921229          1.390551  0.0  0.0   102.0   188.0  46.0     1.0   

   trial  
0    1.0  
1    1.0  
2    1.0

In [6]:
train_data, act_train_labels, id_train_labels = shuffle(train_data, act_train_labels, id_train_labels, random_state=0)

In [7]:
## Here we add an extra dimension to the datasets just to be ready for using with Convolution2D
train_data = np.expand_dims(train_data,axis=3)
print("[INFO] -- Shape of Training Sections:", train_data.shape)
# val_data = np.expand_dims(val_data,axis=3)
# print("[INFO] -- Validation Sections:"+str(val_data.shape))
test_data = np.expand_dims(test_data,axis=3)
print("[INFO] -- Shape of Training Sections:", test_data.shape)

('[INFO] -- Shape of Training Sections:', (60059, 2, 128, 1))
('[INFO] -- Shape of Training Sections:', (13343, 2, 128, 1))


In [8]:
import sys
window = 10 # SSA window == number of components
ssa_train_data = train_data.copy()
# ssa_val_data = val_data.copy()
ssa_test_data = test_data.copy()
ssa_train_0 = []
ssa_train_1 = []
# ssa_val_0 = []
# ssa_val_1 = []
ssa_test_0 = []
ssa_test_1 = []

print("\n Train \n")
for i in range(len(ssa_train_data)):
    ssa_train_0.append(SSA(ssa_train_data[i,0,:,0], window))
    ssa_train_1.append(SSA(ssa_train_data[i,1,:,0], window))
    if(i%100==1):
        sys.stdout.write("\rNow: "+str(np.round(i*100/len(ssa_train_data), 2))+"%")
        sys.stdout.flush()

# print("\n Val \n")
# for i in range(len(ssa_val_data)):
#     ssa_val_0.append(SSA(ssa_val_data[i,0,:,0], window))
#     ssa_val_1.append(SSA(ssa_val_data[i,1,:,0], window))
#     if(i%100==1):
#         sys.stdout.write("\rNow: "+str(np.round(i*100/len(ssa_val_data), 2))+"%")
#         sys.stdout.flush()        

print("\n Test \n")
for i in range(len(ssa_test_data)):
    ssa_test_0.append(SSA(ssa_test_data[i,0,:,0], window))
    ssa_test_1.append(SSA(ssa_test_data[i,1,:,0], window))
    if(i%100==1):
        sys.stdout.write("\rNow: "+str(np.round(i*100/len(ssa_test_data), 2))+"%")
        sys.stdout.flush()


 Train 

Now: 99%
 Test 

Now: 99%

In [12]:
id_history = {}
ep = 32
for num_comps in range(1,11):
    ssa_train_data = train_data.copy()
    ssa_test_data = test_data.copy()
    
    print("With "+str(num_comps)+" components:")
    for i in range(len(ssa_train_data)):
        ssa_train_data[i,0,:,0] = ssa_train_0[i].reconstruct(list(range(0,num_comps)))
        ssa_train_data[i,1,:,0] = ssa_train_1[i].reconstruct(list(range(0,num_comps)))
    for i in range(len(ssa_test_data)):
        ssa_test_data[i,0,:,0] = ssa_test_0[i].reconstruct(list(range(0,num_comps)))
        ssa_test_data[i,1,:,0] = ssa_test_1[i].reconstruct(list(range(0,num_comps)))
    
    height = train_data.shape[1]
    width = train_data.shape[2]

    id_class_numbers = 24
    act_class_numbers = 4
    fm = (2,5)

    print("___________________________________________________")
    ## Callbacks
    eval_metric= "val_acc"    
    #eval_metric= "val_f1_metric"    
    early_stop = keras.callbacks.EarlyStopping(monitor=eval_metric, mode='max', patience = 7)
    filepath="XXID.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor=eval_metric, verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint, early_stop]
    ## Callbacks

    eval_act = Estimator.build(height, width, id_class_numbers, name ="EVAL_ACT", fm=fm, act_func="softmax",hid_act_func="relu")
    eval_act.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc',f1_metric])
    print("Model Size = "+str(eval_act.count_params()))

    eval_act.fit(ssa_train_data, id_train_labels,
                validation_split = .2,
                epochs = ep,
                batch_size = 128,
                verbose = 0,
                class_weight = get_class_weights(np.argmax(id_train_labels,axis=1)),
                callbacks = callbacks_list
               )

    eval_act.load_weights("XXID.best.hdf5")
    eval_act.compile( loss="categorical_crossentropy", optimizer='adam', metrics=['acc',f1_metric])

    result1 = eval_act.evaluate(ssa_test_data, id_test_labels, verbose = 2)
    act_acc = result1[1].round(4)*100
    print("***[RESULT]*** ID Accuracy: "+str(act_acc))

    preds = eval_act.predict(ssa_test_data)
    preds = np.argmax(preds, axis=1)
    conf_mat = confusion_matrix(np.argmax(id_test_labels, axis=1), preds)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print("***[RESULT]*** ID  Confusion Matrix")
    print((np.array(conf_mat).diagonal()).round(3)*100)  

    f1act = f1_score(np.argmax(id_test_labels, axis=1), preds, average=None).mean()
    print("***[RESULT]*** ID Averaged F-1 Score : "+str(f1act))

    id_history[num_comps] = f1act

With 1 components:
___________________________________________________
Model Size = 158040

Epoch 00001: val_acc improved from -inf to 0.35098, saving model to XXID.best.hdf5

Epoch 00002: val_acc improved from 0.35098 to 0.45912, saving model to XXID.best.hdf5

Epoch 00003: val_acc improved from 0.45912 to 0.57642, saving model to XXID.best.hdf5

Epoch 00004: val_acc improved from 0.57642 to 0.61747, saving model to XXID.best.hdf5

Epoch 00005: val_acc improved from 0.61747 to 0.65385, saving model to XXID.best.hdf5

Epoch 00006: val_acc improved from 0.65385 to 0.71528, saving model to XXID.best.hdf5

Epoch 00007: val_acc improved from 0.71528 to 0.76698, saving model to XXID.best.hdf5

Epoch 00008: val_acc improved from 0.76698 to 0.77706, saving model to XXID.best.hdf5

Epoch 00009: val_acc improved from 0.77706 to 0.79537, saving model to XXID.best.hdf5

Epoch 00010: val_acc improved from 0.79537 to 0.81477, saving model to XXID.best.hdf5

Epoch 00011: val_acc improved from 0.8147


Epoch 00004: val_acc improved from 0.65210 to 0.75982, saving model to XXID.best.hdf5

Epoch 00005: val_acc improved from 0.75982 to 0.83908, saving model to XXID.best.hdf5

Epoch 00006: val_acc improved from 0.83908 to 0.88195, saving model to XXID.best.hdf5

Epoch 00007: val_acc improved from 0.88195 to 0.89777, saving model to XXID.best.hdf5

Epoch 00008: val_acc improved from 0.89777 to 0.91434, saving model to XXID.best.hdf5

Epoch 00009: val_acc improved from 0.91434 to 0.92799, saving model to XXID.best.hdf5

Epoch 00010: val_acc improved from 0.92799 to 0.93124, saving model to XXID.best.hdf5

Epoch 00011: val_acc improved from 0.93124 to 0.93606, saving model to XXID.best.hdf5

Epoch 00012: val_acc improved from 0.93606 to 0.94514, saving model to XXID.best.hdf5

Epoch 00013: val_acc improved from 0.94514 to 0.95147, saving model to XXID.best.hdf5

Epoch 00014: val_acc did not improve from 0.95147

Epoch 00015: val_acc improved from 0.95147 to 0.95230, saving model to XXID.be


Epoch 00010: val_acc improved from 0.93307 to 0.95413, saving model to XXID.best.hdf5

Epoch 00011: val_acc did not improve from 0.95413

Epoch 00012: val_acc improved from 0.95413 to 0.95621, saving model to XXID.best.hdf5

Epoch 00013: val_acc did not improve from 0.95621

Epoch 00014: val_acc improved from 0.95621 to 0.96429, saving model to XXID.best.hdf5

Epoch 00015: val_acc did not improve from 0.96429

Epoch 00016: val_acc did not improve from 0.96429

Epoch 00017: val_acc did not improve from 0.96429

Epoch 00018: val_acc did not improve from 0.96429

Epoch 00019: val_acc did not improve from 0.96429

Epoch 00020: val_acc improved from 0.96429 to 0.96637, saving model to XXID.best.hdf5

Epoch 00021: val_acc improved from 0.96637 to 0.97028, saving model to XXID.best.hdf5

Epoch 00022: val_acc did not improve from 0.97028

Epoch 00023: val_acc improved from 0.97028 to 0.97469, saving model to XXID.best.hdf5

Epoch 00024: val_acc improved from 0.97469 to 0.97527, saving model t


Epoch 00015: val_acc did not improve from 0.96603

Epoch 00016: val_acc improved from 0.96603 to 0.96845, saving model to XXID.best.hdf5

Epoch 00017: val_acc did not improve from 0.96845

Epoch 00018: val_acc did not improve from 0.96845

Epoch 00019: val_acc improved from 0.96845 to 0.96945, saving model to XXID.best.hdf5

Epoch 00020: val_acc did not improve from 0.96945

Epoch 00021: val_acc improved from 0.96945 to 0.97261, saving model to XXID.best.hdf5

Epoch 00022: val_acc improved from 0.97261 to 0.97577, saving model to XXID.best.hdf5

Epoch 00023: val_acc did not improve from 0.97577

Epoch 00024: val_acc improved from 0.97577 to 0.97611, saving model to XXID.best.hdf5

Epoch 00025: val_acc did not improve from 0.97611

Epoch 00026: val_acc did not improve from 0.97611

Epoch 00027: val_acc improved from 0.97611 to 0.97702, saving model to XXID.best.hdf5

Epoch 00028: val_acc did not improve from 0.97702

Epoch 00029: val_acc did not improve from 0.97702

Epoch 00030: val_a

In [13]:
id_history

{1: 0.8279729812962838,
 2: 0.8788185662593486,
 3: 0.9011224131632366,
 4: 0.9060514528886765,
 5: 0.9131232100931586,
 6: 0.9383943135784875,
 7: 0.9345153899396502,
 8: 0.9103742050946199,
 9: 0.9112390417002011,
 10: 0.9153658253091729}